In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import DarkNews as dn

from DarkNews import const

from DarkNews.GenLauncher import GenLauncher

from DarkNews import Cfourvec as Cfv

from particle import literals as lp
from pathlib import Path

from ToyAnalysis import analysis_decay as ad
from ToyAnalysis import analysis as an
from ToyAnalysis import fit_functions as fit
from ToyAnalysis.plot_tools import plot_all_rates as MBplotter
from pathlib import Path

from DarkNews.MC import get_merged_MC_output
import scipy
from DarkNews.fourvec import df_dot3
def get_pval(df_prob, ndof=6.7):
    bin_e = np.genfromtxt(f'ToyAnalysis/data/MB_data_release/nue2020/numode/miniboone_binboundaries_nue_lowe.txt')
    NP_MC, _ = np.histogram(df_prob['reco_Enu']*1e3, weights=df_prob['reco_w'], bins=bin_e)
    bw = bin_e[1:]-bin_e[:-1]
    plt.step((bin_e[1:]+bin_e[:-1])/2,NP_MC/bw, where ='mid')
    chi2=fit.chi2_MiniBooNE_2020(NP_MC, NP_MC.sum())
    return scipy.stats.chi2.sf(chi2, 8.7)

def print_summary_of_event(df, col='w_event_rate', summary='Events'):
    coherent = (df['scattering_regime'] == 'coherent')
    pel = (df['scattering_regime'] == 'p-el')
    nel = (df['scattering_regime'] == 'n-el')
    print(f"{summary}: {df[col].sum():.2g} total ({df[col][coherent].sum():.2g} coh : {df[col][pel].sum():.2g} pel : {df[col][nel].sum():.2g} nel)")


In [ ]:
common_kwargs = {'loglevel': 'ERROR', 'HNLtype': 'dirac', 'neval': 1e4}

# TMM
kwargs = {'mu_tr_mu4': 1.4e-6, 'm4': 0.470, 'decay_product': 'photon'}
gen_1 = dn.GenLauncher(experiment='miniboone_fhc', **kwargs, **common_kwargs)
gen_1_dirt = dn.GenLauncher(experiment='miniboone_fhc_dirt', **kwargs, **common_kwargs)
genmuB_1 = dn.GenLauncher(experiment='microboone', **kwargs, **common_kwargs)
genmuB_1_dirt = dn.GenLauncher(experiment='microboone_dirt', **kwargs, **common_kwargs)
df_1 = gen_1.run()
df_1_dirt = gen_1_dirt.run()
df_1_muB = genmuB_1.run()
df_1_muB_dirt = genmuB_1_dirt.run()

# Vectors
M4 = 0.107
M5 = 0.150
kwargs = {'d_mu5': 0.0002, 'd_45': 1, 'dprotonV': 3.4e-4*const.eQED, 'deV': 3.4e-4*const.eQED, 'mzprime': 0.03, 'm4': M4, 'm5': M5}
gen_2 = dn.GenLauncher(experiment='miniboone_fhc', **kwargs, **common_kwargs)
gen_2_dirt = dn.GenLauncher(experiment='miniboone_fhc_dirt', **kwargs, **common_kwargs)
gen_2_muB = dn.GenLauncher(experiment='microboone', **kwargs, **common_kwargs)
gen_2_muB_dirt = dn.GenLauncher(experiment='microboone_dirt', **kwargs, **common_kwargs)
df_2 = gen_2.run()
df_2_dirt = gen_2_dirt.run()
df_2_muB = gen_2_muB.run()
df_2_muB_dirt = gen_2_muB_dirt.run()

kwargs = {'d_mu5': 0.0002, 'd_45': 1, 'dprotonV': 3.4e-4*const.eQED, 'deV': 3.4e-4*const.eQED, 'mzprime': 0.03, 'm4': M4, 'm5': M5}
kwargs = {'UD4': 1/np.sqrt(2), 'UD5': 1/np.sqrt(2), 'Umu5': 1e-3, 'Umu4': 1e-3, 'gD': 2, 'epsilon': 0.75e-2, 'mzprime': 1.25, 'm4': M4, 'm5': M5}
gen_3 = dn.GenLauncher(experiment='miniboone_fhc',  **kwargs, **common_kwargs)
gen_3_dirt = dn.GenLauncher(experiment='miniboone_fhc_dirt',  **kwargs, **common_kwargs)
gen_3_muB = dn.GenLauncher(experiment='microboone', **kwargs, **common_kwargs)
gen_3_muB_dirt = dn.GenLauncher(experiment='microboone_dirt', **kwargs, **common_kwargs)
df_3 = gen_3.run()
df_3_dirt = gen_3_dirt.run()
df_3_muB = gen_3_muB.run()
df_3_muB_dirt = gen_3_muB_dirt.run()

# Scalars
M4 = 0.140
M5 = 0.2
theta_e = 2.5e-3
theta_numu = 4e-3
theta_nucleon = 4e-3
kwargs = {'mzprime': None, 'mhprime': 1.0, 'm4': M4, 'm5': M5, 's_mu5': theta_numu, 's_45': 1.0,  'deS': theta_e, 'dprotonS': theta_nucleon, 'dneutronS': theta_nucleon}
gen_4 = dn.GenLauncher(experiment='miniboone_fhc', **kwargs, **common_kwargs)
gen_4_dirt = dn.GenLauncher(experiment='miniboone_fhc_dirt', **kwargs, **common_kwargs)
gen_4_muB = dn.GenLauncher(experiment='microboone', **kwargs, **common_kwargs)
gen_4_muB_dirt = dn.GenLauncher(experiment='microboone_dirt', **kwargs, **common_kwargs)
df_4 = gen_4.run()
df_4_dirt = gen_4_dirt.run()
df_4_muB = gen_4_muB.run()
df_4_muB_dirt = gen_4_muB_dirt.run()


theta_e = 0.8e-3
theta_numu = 1e-3
theta_nucleon = 1.4e-4
kwargs = {'mzprime': None, 'mhprime': 0.03, 'm4': M4, 'm5': M5, 's_mu5': theta_numu, 's_45': 1.0,  'deS': theta_e, 'dprotonS': theta_nucleon, 'dneutronS': theta_nucleon}
gen_5 = dn.GenLauncher(experiment='miniboone_fhc', **kwargs, **common_kwargs)
gen_5_dirt = dn.GenLauncher(experiment='miniboone_fhc_dirt', **kwargs, **common_kwargs)
gen_5_muB = dn.GenLauncher(experiment='microboone',**kwargs, **common_kwargs)
gen_5_muB_dirt = dn.GenLauncher(experiment='microboone_dirt',**kwargs, **common_kwargs)
df_5 = gen_5.run()
df_5_dirt = gen_5_dirt.run()
df_5_muB = gen_5_muB.run()
df_5_muB_dirt = gen_5_muB_dirt.run()


In [ ]:
### Performing analysis 

In [ ]:
my_dfs = [df_1, df_2, df_3, df_4, df_5]
my_dfs_dirt = [df_1_dirt, df_2_dirt, df_3_dirt, df_4_dirt, df_5_dirt]
my_dfs_muB = [df_1_muB, df_2_muB, df_3_muB, df_4_muB, df_5_muB]
my_dfs_muB_dirt = [df_1_muB_dirt, df_2_muB_dirt, df_3_muB_dirt, df_4_muB_dirt, df_5_muB_dirt]

##########
# MiniBooNE -- smear, signal selection and filter decays
my_dfs_prob = []
my_dfs_dirt_prob = []
final_dfs = []
for df, dfd in zip(my_dfs, my_dfs_dirt):
    if df.attrs['model'].mu_tr_mu4 > 0:
        dfp   = an.compute_spectrum(ad.decay_selection(df, l_decay_proper_cm=df.attrs['N4_ctau0'], experiment='miniboone'), EVENT_TYPE='photon')
        dfp_d = an.compute_spectrum(ad.decay_selection(dfd, l_decay_proper_cm=dfd.attrs['N4_ctau0'], experiment='miniboone'), EVENT_TYPE='photon')
    else:
        dfp   = an.compute_spectrum(ad.decay_selection(df, l_decay_proper_cm=df.attrs['N5_ctau0'], experiment='miniboone'), EVENT_TYPE='both')
        dfp_d = an.compute_spectrum(ad.decay_selection(dfd, l_decay_proper_cm=dfd.attrs['N5_ctau0'], experiment='miniboone'), EVENT_TYPE='both')
    my_dfs_prob.append(dfp)
    my_dfs_dirt_prob.append(dfp_d)
    final_dfs.append(get_merged_MC_output(dfp,dfp_d))

##########
# MicroBooNE -- smear, signal selection and filter decays
my_dfs_muB_prob = []
my_dfs_muB_dirt_prob = []
final_dfs_muB = []
for df, dfd in zip(my_dfs_muB, my_dfs_muB_dirt):
    if df.attrs['model'].mu_tr_mu4 > 0:
        dfp   = an.compute_spectrum(ad.decay_selection(df, l_decay_proper_cm=df.attrs['N4_ctau0'], experiment='microboone'), EVENT_TYPE='photon', EXP='microboone')
        dfp_d = an.compute_spectrum(ad.decay_selection(dfd, l_decay_proper_cm=dfd.attrs['N4_ctau0'], experiment='microboone'), EVENT_TYPE='photon', EXP='microboone')
    else:
        dfp   = an.compute_spectrum(ad.decay_selection(df, l_decay_proper_cm=df.attrs['N5_ctau0'], experiment='microboone'), EVENT_TYPE='both', EXP='microboone')
        dfp_d = an.compute_spectrum(ad.decay_selection(dfd, l_decay_proper_cm=dfd.attrs['N5_ctau0'], experiment='microboone'), EVENT_TYPE='both', EXP='microboone')
    my_dfs_muB_prob.append(dfp)
    my_dfs_muB_dirt_prob.append(dfp_d)
    final_dfs_muB.append(get_merged_MC_output(dfp,dfp_d))

In [2]:
### Some tests

In [3]:
df = final_dfs[2]
dirt_mask = (df['pos_scatt','3']<-610)
tmax = 100
_ = plt.hist(df['pos_decay','0'][dirt_mask]*1e9, weights=df['reco_w'][dirt_mask], bins=np.linspace(0,tmax, 21), histtype='bar', color='black',
            label='dirt')
ht,_,_ = plt.hist(df['pos_decay','0']*1e9, weights=df['reco_w'], bins=np.linspace(0,tmax, 21), histtype='step', color='black',
            label='total')

plt.vlines(10,0,1e3, colors='black', ls='--')
plt.legend(loc='upper right', frameon=False)
plt.xlabel("t (ns)")
plt.ylabel("events")
plt.xlim(0,tmax)
plt.ylim(0,ht[0]*1.1)


NameError: name 'final_dfs' is not defined

In [4]:
## Printing summary for MiniBooNE benchmarks 

for i, (df, df_mub) in enumerate(zip(final_dfs,final_dfs_muB)):
    if i>0:
        print(f'\nBenchmark {i+1}: mzprime = {df.attrs["model"].mzprime}, mhprime = {df.attrs["model"].mhprime}')
        print(f"{df.attrs['N5_ctau0']}")

    else:
        print(f'\nBenchmark {i+1}: transition mag moment')
        print(f"ctau0 = {df.attrs['N4_ctau0']:2g} cm")

    print_summary_of_event(df, col='w_pre_decay',summary='Raw events at MiniBooNE')
    print_summary_of_event(df, col='reco_w', summary='Selected events at MiniBooNE')
    print(f'MiniBooNE p-value: {get_pval(df)}')
    print_summary_of_event(df_mub, col='w_pre_decay',summary='Raw events at MicroBooNE')

NameError: name 'final_dfs' is not defined

In [ ]:
# Im missing the miniboone plots for TMM...


MBplotter(final_dfs[1], 'BP light vector', title='$m_5 = 150$ MeV,  $\Delta_{54} = 0.4$,  $m_{Z^\prime} = 30$ MeV')
MBplotter(final_dfs[2], 'BP heavy vector', title='$m_5 = 150$ MeV,  $\Delta_{54} = 0.4$,  $m_{Z^\prime} = 1.25$ GeV')
MBplotter(final_dfs[3], 'BP light scalar', title='$m_5 = 200$ MeV,  $\Delta_{54} = 1$,  $m_{h^\prime} = 30$ MeV')
MBplotter(final_dfs[4], 'BP heavy scalar', title='$m_5 = 200$ MeV,  $\Delta_{54} = 1$,  $m_{h^\prime} = 1.0$ MeV')